In [1]:
import spacy
#from spacy import displacy
import explacy # simple parser visualization
import pandas as pd
import glob
from importlib import reload

In [2]:
nlp = spacy.load('en')

In [3]:
reload(explacy)

<module 'explacy' from '/root/w266_final/explacy.py'>

In [4]:
# suprisingly becomes child of was, but maybe this is more correct
# text = 'The salad was surprisingly tasty.'
# explacy.print_parse_info(nlp, text)

In [5]:
# this looks right... 
# text = 'The salad was unexpectedly tasty.'
# explacy.print_parse_info(nlp, text)

In [6]:
# bring 1 json into a dataframe for analysis
files = sorted(glob.glob('layer1.json.[0].valid.gz'))
df = pd.concat([pd.read_pickle(fp) for fp in files], ignore_index=True)
df.shape

(99999, 9)

In [7]:
# let's filter out epicurious recipes with less than 10 steps
keep_columns = ['id', 'partition', 
                'collection', 'title', 
                'ingr_count', 'instr_count', 
                'ingredients', 'instructions']

epicurious = df[(df.collection == 'www.epicurious.com') 
   & (df.instr_count<=5)][keep_columns].sort_values(by=['instr_count'], ascending=False)

# train, val, test counts
epicurious.groupby('partition').size()

partition
test     111
train    489
val      110
dtype: int64

In [8]:
# for simplicity, let's look at recipes with just less than 5 steps 
test_recipes = epicurious[(epicurious.instr_count<=5)].head(10).index

In [9]:
# Garlic Broth with Sage and Parsley, 21658
# Grilled Corn on the Cob, 96647

# for recipe in test_recipes:
#     # title
#     print('\n' + epicurious['title'][recipe] + '\n')
    
#     # ingredients
#     for index, each in enumerate(epicurious['ingredients'][recipe]):
#         print('Ingredient', index+1, ':', list(each.values())[0])
    
#     # instructoins    
#     for index, each in enumerate(epicurious['instructions'][recipe]):
#         if index == 0:
#             print('Step', index+1, ':', list(each.values())[0])

In [10]:
# spaCy parameters under Token:
# Text: The original word text.
# Lemma: The base form of the word.
# POS: The simple part-of-speech tag.
# Tag: The detailed part-of-speech tag.
# Dep: Syntactic dependency, i.e. the relation between tokens.
# Shape: The word shape – capitalisation, punctuation, digits.
# is alpha: Is the token an alpha character?
# is stop: Is the token part of a stop list, i.e. the most common words of the language?

for recipe in test_recipes:
    # title
    print('\n' + epicurious['title'][recipe].upper() + '\n')
    
    # instructoins    
    for index, each in enumerate(epicurious['instructions'][recipe]):
        if index == 0:
            instruction = list(each.values())[0]
            print('1st step:', instruction)
            #displacy.render(nlp(instruction), style='ent', jupyter=True)
            # print parser tree for each step
            explacy.print_parse_info(nlp, instruction)


CONFETTI SALAD WITH RANCH DRESSING

1st step: Combine all vegetables in large bowl.
Dep tree   Token      Dep   Tag Pos  
────────── ────────── ───── ─── ─────
┌┬──┬───── Combine    ROOT  VB  VERB 
││  │  ┌─► all        det   DT  DET  
││  └─►└── vegetables dobj  NNS NOUN 
│└─►┌───── in         prep  IN  ADP  
│   │  ┌─► large      amod  JJ  ADJ  
│   └─►└── bowl       pobj  NN  NOUN 
└────────► .          punct .   PUNCT

MANGO CURRY TRUFFLES

1st step: Follow recipe for bittersweet chocolate truffle base , stirring curry powder into hot cream (off heat); let stand 2 minutes.
Dep tree             Token       Dep      Tag   Pos  
──────────────────── ─────────── ──────── ───── ─────
                 ┌─► Follow      compound VB    VERB 
┌─►┌┬─┬┬─────────┴── recipe      nsubj    NN    NOUN 
│  ││ │└─►┌───────── for         prep     IN    ADP  
│  ││ │   │  ┌─────► bittersweet amod     JJ    ADJ  
│  ││ │   │  │   ┌─► chocolate   compound NN    NOUN 
│  ││ │   │  │┌─►└── truffle     comp

In [ ]:
# for each in (epicurious['ingredients'].head(1)):
#     for k, v in enumerate(each):
#         doc = nlp(list(v.values())[0])
#         print(list(v.values())[0])
#         for chunk in doc.noun_chunks:
#             print(chunk.root)

In [ ]:
# import benepar
# benepar.download('benepar_en')

In [ ]:
# from benepar.spacy_plugin import BeneparComponent

In [ ]:
# nlp = spacy.load('en')
# nlp.add_pipe(BeneparComponent('benepar_en'))

In [ ]:
# doc = nlp("Bring some light chicken broth to a boil with a few leaves of fresh sage.")
# sent = list(doc.sents)[0]

In [ ]:
# for each in enumerate (sent._.children):
#     print(each)

In [ ]:
# type(sent._.children)

In [ ]:
# for each in enumerate (sent._.children):
#     # go from the start to the end of each span, returning each token in the sentence
#     # combine each token using join()
#     print(each.parse_string)